In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl


In [2]:
!pip install transformers datasets evaluate accelerate peft trl bitsandbytes
!pip install nvidia-ml-py3
!pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia

In [3]:
!git clone https://github.com/jessenb16/Finetuning-with-LoRA.git
%cd Finetuning-with-LoRA

Cloning into 'Finetuning-with-LoRA'...
remote: Enumerating objects: 1055, done.
remote: Counting objects: 100% (1055/1055), done.
remote: Compressing objects: 100% (975/975), done.
remote: Total 1055 (delta 79), reused 1044 (delta 68), pack-reused 0 (from 0)
Receiving objects: 100% (1055/1055), 4.84 MiB | 26.48 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/kaggle/working/Finetuning-with-LoRA


In [4]:
pkl_file = "/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl"
output_direct = '/kaggle/working/output'
os.makedirs(output_direct, exist_ok=True)

In [5]:
import torch.optim as optim
import torchvision.transforms as transforms
from src.model import setup_lora_model
from src.train import get_datasets, train
from src.evaluate import run_inference, get_unlabelled_dataset
from peft import LoraConfig
from transformers import TrainingArguments, RobertaTokenizer

2025-04-21 16:06:07.565128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745251567.756986      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745251567.808708      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
#Lora Configurations

loraConfig = LoraConfig(
    r=22,  # Rank of the LoRA layers
    lora_alpha=44,  # Scaling factor for LoRA
    lora_dropout=0.1,  # Dropout rate for LoRA layers
    bias="none",  # Bias handling in LoRA layers
    task_type="SEQ_CLS",  # Task type for sequence classification
    target_modules=['value']
)


training_args = TrainingArguments(
        output_dir = output_direct,
        report_to="none",
        eval_strategy = "steps",
        logging_steps = 100,
        learning_rate = 5e-5,
        num_train_epochs = 1,
        max_steps = 1200,
        warmup_steps = 100,
        save_steps = 100,
        save_total_limit=3,
        dataloader_num_workers = 4,
        per_device_train_batch_size = 16,
        per_device_eval_batch_size = 64,
        optim = "adamw_torch",
        gradient_checkpointing = False,
        gradient_checkpointing_kwargs = {'use_reetrant': True}
)

In [12]:
try:
    model, id2label = setup_lora_model(loraConfig)
except AssertionError as e:
    print(f'Failed to create model: {e}')
    exit(1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 999,172 || all params: 125,647,880 || trainable%: 0.7952


In [13]:
try:
    train_dataset, eval_dataset, data_collator = get_datasets()
except (ValueError, TypeError) as e:
    print(f'Training failed: {e}')
    exit(1)

Tokenized dataset features: ['labels', 'input_ids', 'attention_mask']


In [14]:
try:
    train(model, train_dataset, eval_dataset, data_collator, training_args, id2label)
except (ValueError, TypeError) as e:
    print(f'Training failed: {e}')
    exit(1)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Accuracy
100,1.377000,1.328592,0.681250
200,0.684600,0.336248,0.893750
300,0.378800,0.342331,0.890625
400,0.305900,0.334563,0.898438
500,0.303000,0.325297,0.906250
600,0.298400,0.326675,0.898438
700,0.324100,0.315827,0.907813
800,0.314500,0.305166,0.904687
900,0.307100,0.309286,0.907813
1000,0.283300,0.301800,0.909375


In [10]:
try:
    test_dataset = get_unlabelled_dataset(pkl_file)
    run_inference(model, test_dataset, output_direct, data_collator)
except (ValueError, TypeError) as e:
    print(f'Training failed: {e}')
    exit(1)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

100%|██████████| 1000/1000 [00:59<00:00, 16.75it/s]

Inference complete. Predictions saved to /kaggle/working/output
